In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from scipy.stats import gamma,exponweib
from scipy.optimize import fsolve,brentq
from tqdm import tqdm
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.plotting import figure, show
import time


# Constant Infectivity Sims

In [32]:
def SIRSelke_constant_threshold_reduction(n,m,Q,I,lmb,K,D,B):    
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    pulsed = False
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    active_tau = [np.inf]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    

    next_event_tau = {"infection": (Q[0]-A)/((lmb/n)*len(infected_indices)),
                      "recovery": min(active_tau),
                      "start intervention": np.inf,
                      "end intervention": np.inf}
    next_event_i = {"infection": m,
                    "recovery": active_tau.index(next_event_tau["recovery"])}
    
    
    
    effective_lmb = lmb

    
    
    
    while len(infected_indices)>0:
        dt = min(next_event_tau.values())
        active_tau = [tau-dt for tau in active_tau] 
        A+= (effective_lmb/n)*dt*len(infected_indices)
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "start intervention":
            #print("pulse")
            effective_lmb = lmb/B
            next_event_tau["start intervention"] = np.inf
            pulsed = True

            
        elif next_event_type ==  "end intervention":
            #print("unpulse")
            effective_lmb = lmb
            next_event_tau["end intervention"] = np.inf
 
        
        elif next_event_type ==  "recovery":
            #print("Recovery")
            t_recov[next_event_i["recovery"]] = t
            recovered_indices.append(next_event_i["recovery"])
            infected_indices.remove(next_event_i["recovery"])
            active_tau[next_event_i["recovery"]] = np.inf

            
        
            
        elif next_event_type ==  "infection":
            #print("Infection")     
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            active_tau[next_event_i["infection"]]  = I[next_event_i["infection"]]
            next_event_i["infection"] += 1
            if next_event_i["infection"]==n:
                next_event_tau["infection"]= np.inf
            
            
        
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
                next_event_tau["infection"] = (Q[next_event_i["infection"]-m]-A)/((effective_lmb/n)*len(infected_indices))
        next_event_tau["recovery"] = min(active_tau)
        next_event_i["recovery"] = active_tau.index(next_event_tau["recovery"])
        next_event_tau["end intervention"] = next_event_tau["end intervention"] -dt
        if len(infected_indices)+len(recovered_indices) == K and not pulsed:
            next_event_tau["end intervention"] = D
            next_event_tau["start intervention"] = 0
        

                
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        
        #if A_prev>A:
        #    print("A =",A) 
        #print(active_tau)
        #print("")

        ##input("t="+str(t)+"\n")

    return(t_v,A_v,I_v,t_inf,t_recov,recovered_indices)
            


            

In [84]:
def SEIRSelke_constant_reduction(n,m,Q,E,I,lmb,T,D,B):    
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [0]
    E_v = [m]
    
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    exposed_indices = [k for k in range(m)]
    infected_indices = []
    recovered_indices = []
    
    active_exposed_tau = [np.inf]*(n+m)
    active_tau = [np.inf]*(n+m)
    
    for k in exposed_indices:
        active_exposed_tau[k] = E[k]
    

    next_event_tau = {"exposure":np.inf,
                      "infection":min(active_exposed_tau),
                      "recovery": min(active_tau),
                      "start intervention": T,
                      "end intervention": T+D}
    next_event_i = {"exposure": m,
                    "infection":active_exposed_tau.index(next_event_tau["infection"]),
                    "recovery": active_tau.index(next_event_tau["recovery"])}
    
    
    
    effective_lmb = lmb

    
    
    
    while len(exposed_indices)+len(infected_indices)>0:
       
        dt = min(next_event_tau.values())
        
        active_tau = [tau-dt for tau in active_tau] 
        active_exposed_tau = [tau-dt for tau in active_exposed_tau] 
        
        A+= (effective_lmb/n)*dt*len(infected_indices)
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "start intervention":
            #print("pulse")
            effective_lmb = lmb/B
            next_event_tau["start intervention"] = np.inf

            
        elif next_event_type ==  "end intervention":
            #print("unpulse")
            effective_lmb = lmb
            next_event_tau["end intervention"] = np.inf
 
        elif next_event_type == "exposure":
            #print("Exposure")
            exposed_indices.append(next_event_i["exposure"])
            susceptible_indices.remove(next_event_i["exposure"])
            active_exposed_tau[next_event_i["exposure"]]  = I[next_event_i["exposure"]]
            next_event_i["exposure"] += 1
            if len(susceptible_indices)==0:
                next_event_tau["exposure"]= np.inf
        
        elif next_event_type ==  "infection":
            #print("Infection")     
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            exposed_indices.remove(next_event_i["infection"])
            active_tau[next_event_i["infection"]]  = I[next_event_i["infection"]]
            active_exposed_tau[next_event_i["infection"]] = np.inf
            if len(exposed_indices)==0:
                next_event_tau["infection"]= np.inf    
        
        elif next_event_type ==  "recovery":
            #print("Recovery")
            t_recov[next_event_i["recovery"]] = t
            recovered_indices.append(next_event_i["recovery"])
            infected_indices.remove(next_event_i["recovery"])
            active_tau[next_event_i["recovery"]] = np.inf

            
        
            
       
            
            
        
        if len(susceptible_indices)!=0 and len(infected_indices)!=0:
                next_event_tau["exposure"] = (Q[next_event_i["exposure"]-m]-A)/((effective_lmb/n)*len(infected_indices))
                
        next_event_tau["infection"] = min(active_exposed_tau)
        next_event_i["infection"] = active_exposed_tau.index(next_event_tau["infection"])                
        next_event_tau["recovery"] = min(active_tau)
        next_event_i["recovery"] = active_tau.index(next_event_tau["recovery"])        
        next_event_tau["end intervention"] = next_event_tau["end intervention"] -dt
        next_event_tau["start intervention"] = next_event_tau["start intervention"] -dt
                
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        E_v.append(len(exposed_indices))
        
        #if A_prev>A:
        #    print("A =",A) 
        #print(active_tau)
        #print("")

        #print("t=",t,"\n")

    return(t_v,A_v,I_v,E_v,t_inf,t_recov,recovered_indices)

In [56]:
def SIRSelke_constant_reduction(n,m,Q,I,lmb,T,D,B):    
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    active_tau = [np.inf]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    

    next_event_tau = {"infection": (Q[0]-A)/((lmb/n)*len(infected_indices)),
                      "recovery": min(active_tau),
                      "start intervention": T,
                      "end intervention": T+D}
    next_event_i = {"infection": m,
                    "recovery": active_tau.index(next_event_tau["recovery"])}
    
    
    
    effective_lmb = lmb

    
    
    
    while len(infected_indices)>0:
        
        dt = min(next_event_tau.values())
        
        active_tau = [tau-dt for tau in active_tau] 
        A+= (effective_lmb/n)*dt*len(infected_indices)
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "start intervention":
            #print("pulse")
            effective_lmb = lmb/B
            next_event_tau["start intervention"] = np.inf

            
        elif next_event_type ==  "end intervention":
            #print("unpulse")
            effective_lmb = lmb
            next_event_tau["end intervention"] = np.inf
 
        
        elif next_event_type ==  "recovery":
            #print("Recovery")
            t_recov[next_event_i["recovery"]] = t
            recovered_indices.append(next_event_i["recovery"])
            infected_indices.remove(next_event_i["recovery"])
            active_tau[next_event_i["recovery"]] = np.inf

            
        
            
        elif next_event_type ==  "infection":
            #print("Infection")     
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            active_tau[next_event_i["infection"]]  = I[next_event_i["infection"]]
            next_event_i["infection"] += 1
            if next_event_i["infection"]==n:
                next_event_tau["infection"]= np.inf
            
            
        
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
                next_event_tau["infection"] = (Q[next_event_i["infection"]-m]-A)/((effective_lmb/n)*len(infected_indices))
        next_event_tau["recovery"] = min(active_tau)
        next_event_i["recovery"] = active_tau.index(next_event_tau["recovery"])        
        next_event_tau["end intervention"] = next_event_tau["end intervention"] -dt
        next_event_tau["start intervention"] = next_event_tau["start intervention"] -dt
                
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        
        #if A_prev>A:
        #    print("A =",A) 
        #print(active_tau)
        #print("")

        #print("t=",t,"\n")

    return(t_v,A_v,I_v,t_inf,t_recov,recovered_indices)
            


            

In [93]:
def SIRSelke_constant_pulse(n,m,Q,I,lmb,T,P,p):
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    active_tau = [np.inf]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    

    next_event_tau = {"infection": (Q[0]-A)/((lmb/n)*len(infected_indices)),
                      "recovery": min(active_tau),
                      "start intervention": T}
    next_event_i = {"infection": m,
                    "recovery": active_tau.index(next_event_tau["recovery"])}
    
    
    
    effective_lmb = lmb

    
    
    
    while len(infected_indices)>0:
        
        dt = min(next_event_tau.values())
        
        active_tau = [tau-dt for tau in active_tau] 
        A+= (effective_lmb/n)*dt*len(infected_indices)
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "start intervention":
            #print("pulse")
            for i in infected_indices:
                if P[i]<p:
                    indices_spotted.append(i)
                    
                    active_tau[i] = np.inf
                    
            indices_spotted = []       
            for i in indices_spotted:
                recovered_indices.append(i)
                infected_indices.remove(i)
                t_recov[i] = t
            next_event_tau["start intervention"] = np.inf

            
 
        
        elif next_event_type ==  "recovery":
            #print("Recovery")
            t_recov[next_event_i["recovery"]] = t
            recovered_indices.append(next_event_i["recovery"])
            infected_indices.remove(next_event_i["recovery"])
            active_tau[next_event_i["recovery"]] = np.inf

            
        
            
        elif next_event_type ==  "infection":
            #print("Infection")     
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            active_tau[next_event_i["infection"]]  = I[next_event_i["infection"]]
            next_event_i["infection"] += 1
            if next_event_i["infection"]==n+1:
                next_event_tau["infection"]= np.inf
            
            
        
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
                next_event_tau["infection"] = (Q[next_event_i["infection"]-m]-A)/((effective_lmb/n)*len(infected_indices))
        next_event_tau["recovery"] = min(active_tau)
        next_event_i["recovery"] = active_tau.index(next_event_tau["recovery"])    
        next_event_tau["start intervention"] = next_event_tau["start intervention"] -dt
                
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        
        #if A_prev>A:
        #    print("A =",A) 
        #print(active_tau)
        #print("")

        #print("t=",t,"\n")

    return(t_v,A_v,I_v,t_inf,t_recov,recovered_indices)
            
            

# Weibull Infectivity Sim

In [260]:
def SIRSelke_Weibull(n,m,Q,lmb,a,b):
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    
    t_inf = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    
    
    f = lambda x: Q[0] - (lmb/n)*sum([1-np.exp(-((x-t_inf[k])/b)**a) for k in infected_indices])
    if Q[0] - (lmb/n)*len(infected_indices)>0.001:
        return(t_v,A_v,I_v,t_inf,infected_indices)
    else:
        next_infection_time = fsolve(f,t+b*((a-1)/a)**(1/a))[0]
    
    next_event_tau = {"infection": next_infection_time}
    next_event_i = {"infection": m}    
    
    
    while len(infected_indices)>0:
        start = time.time()
        dt = min(next_event_tau.values())
        A+= (lmb/n)*sum([np.exp(-((t-t_inf[k])/b)**a)-np.exp(-((t+dt-t_inf[k])/b)**a) for k in infected_indices])#force up until current event
        #print(A)
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "infection":
            #print("Infection")  
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            susceptible_indices.remove(next_event_i["infection"])
            next_event_i["infection"] += 1
            if len(susceptible_indices)==0:
                next_event_tau["infection"]= np.inf
                return(t_v,A_v,I_v,t_inf,infected_indices)
            
            
        
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
            f = lambda x: Q[next_event_i["infection"]-m] - (lmb/n)*sum([np.exp(-((t-t_inf[k])/b)**a)-np.exp(-((x-t_inf[k])/b)**a) for k in infected_indices])-A
            if Q[next_event_i["infection"]-m] - (lmb/n)*sum([np.exp(-((t-t_inf[k])/b)**a) for k in infected_indices]) -A>0.001:
                
                return(t_v,A_v,I_v,t_inf,infected_indices)
                
            else:
                next_infection_time = np.inf
                i=1
                while f(next_infection_time)<-0.001 :
                    j=1
                    while f(next_infection_time)<-0.001 and j<=9:
                        next_infection_time = fsolve(f,t+(j/(10**i)))[0]
                        #print("check",f(next_infection_time))
                        next_event_tau["infection"] = next_infection_time-t
                        j+=1
                    i+=1
            #print("check", (lmb/n)*sum([np.exp(-((t-t_inf[k])/b)**a)-np.exp(-((next_infection_time-t_inf[k])/b)**a)for k in infected_indices]))           
                
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        #if A_prev>A:
        #    print("A =",A) 
        #print(active_tau)
        #print("")
        end = time.time()
        print(end-start)
        #print("t=",t,"\n")

    return(t_v,A_v,I_v,t_inf,infected_indices)

# General Infectivity Profile Sims

In [184]:
def SIRSelke_general(n,m,Q,inf_cdf,R0):
    A=0
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    
    t_inf = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    
    
    f = lambda x: Q[0] - (R0/n)*sum([inf_cdf(0,x) for k in infected_indices])
    if f(np.inf)>0:
        return(t_v,A_v,I_v,t_inf,infected_indices)
    else:
        next_infection_time = brentq(f,t,t+20)
    
    next_event_tau = {"infection": next_infection_time}
    next_event_i = {"infection": m}    
    
    
    while len(infected_indices)>0:

        dt = min(next_event_tau.values())
        A+= (R0/n)*sum([inf_cdf(t-t_inf[k],t+dt-t_inf[k]) for k in infected_indices])#force up until current event
        t+=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt][0]

        if next_event_type ==  "infection":
            #print("Infection")  
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            susceptible_indices.remove(next_event_i["infection"])
            next_event_i["infection"] += 1
            if len(susceptible_indices)==0:
                next_event_tau["infection"]= np.inf
                return(t_v,A_v,I_v,t_inf,infected_indices)
            
            
        
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
            f = lambda x: Q[next_event_i["infection"]-m]-(R0/n)*sum([inf_cdf(t-t_inf[k],x-t_inf[k]) for k in infected_indices]) -A
            if f(np.inf)>0:
                #print(A)
                return(t_v,A_v,I_v,t_inf,infected_indices)
                
            else:
                next_infection_time = brentq(f,t,t+200)
                next_event_tau["infection"] = next_infection_time-t
                
            
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        
        #print("t=",t,"\n")

    return(t_v,A_v,I_v,t_inf,infected_indices)

In [177]:
def SIRSelke_general_pulse(n,m,Q,P,inf_cdf,R0,symp_tau,test_pdf,symp_thresh):
    ################################################################################################################################################
    #Initialise variables
    A=0
    t = 0
    
    t_v = [0]
    A_v = [0]
    I_v = [m]
    
    t_inf = [0]*(n+m)
    
    pulsed = False

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    symptoms_indices = []
    recovered_indices = []
    
    active_tau = np.array([np.inf]*(n+m))
    for k in infected_indices:
        active_tau[k] = 0
    
    ################################################################################################################################################
    #Calculate next event times and indices    
    f = lambda x: Q[0] - (R0/n)*sum([inf_cdf(0,x) for k in infected_indices])
    if f(np.inf)>0:
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        return(t_v,A_v,I_v,t_inf,len(infected_indices)+len(recovered_indices))
    else:
        next_infection_time = brentq(f,t,t+20)
    
    next_symptom_start_tau = min([symp_tau-tau if (tau<symp_tau) else np.inf for tau in active_tau])
    next_symptom_end_tau = min([symp_tau+7-tau if (tau<symp_tau+7) else np.inf for tau in active_tau])
    next_symptom_start_i = np.where(active_tau == symp_tau -next_symptom_start_tau)[0][0]
    next_symptom_end_i = np.where(active_tau == symp_tau+7 -next_symptom_end_tau)[0][0]

    
    next_event_tau = {"infection": next_infection_time,
                      "start symptoms": next_symptom_start_tau,
                      "end symptoms": next_symptom_end_tau,
                      "pulse": np.inf}
    next_event_i = {"infection": m,
                    "start symptoms": next_symptom_start_i,
                    "end symptoms": next_symptom_end_i}    

    
    while len(infected_indices)>0:
        ################################################################################################################################################
        #Update time and force of infection and decide next event type
        dt = min(next_event_tau.values())
        A+= (R0/n)*sum([inf_cdf(t-t_inf[k],t+dt-t_inf[k]) for k in infected_indices])#force up until current event
        t+=dt
        for k in infected_indices:
            active_tau[k] +=dt
        next_event_type = [i for i in next_event_tau.keys() if next_event_tau[i] == dt]
        ##print([i for i in next_event_tau.keys() if next_event_tau[i] == dt])
        ################################################################################################################################################
        #Carry out event
        if "infection" in next_event_type  :
            t_inf[next_event_i["infection"]] = t
            infected_indices.append(next_event_i["infection"])
            susceptible_indices.remove(next_event_i["infection"])
            active_tau[next_event_i["infection"]] = 0
            next_event_i["infection"] += 1
            if len(susceptible_indices)==0:
                next_event_tau["infection"]= np.inf
                t_v.append(t)
                A_v.append(A)
                I_v.append(len(infected_indices))
                return(t_v,A_v,I_v,t_inf,len(infected_indices)+len(recovered_indices))
            
        if "start symptoms" in next_event_type :
            symptoms_indices.append(next_event_i["start symptoms"])
            
        
        if  "end symptoms" in next_event_type:
            symptoms_indices.remove(next_event_i["end symptoms"])
            
        if "pulse" in next_event_type :
            for k in infected_indices:
                p = test_pdf(active_tau[k])
                if p>P[k]:
                    infected_indices.remove(k)
                    recovered_indices.append(k)
                    if k in symptoms_indices:
                        symptoms_indices.remove(k)
                    active_tau[k] = np.inf
            next_event_tau["pulse"] = np.inf
            pulsed = True
                    
        ################################################################################################################################################
        #Update next event times
        if next_event_tau["infection"] <np.inf and len(infected_indices)!=0:
            f = lambda x: Q[next_event_i["infection"]-m]-(R0/n)*sum([inf_cdf(t-t_inf[k],x-t_inf[k]) for k in infected_indices]) -A
            if f(np.inf)>0:
                t_v.append(t)
                A_v.append(A)
                I_v.append(len(infected_indices))
                return(t_v,A_v,I_v,t_inf,len(infected_indices)+len(recovered_indices))
                
            else:
                next_infection_time = brentq(f,t,t+200)
                next_event_tau["infection"] = next_infection_time-t
        
        
        
        
        
        
        
        if min(active_tau)<symp_tau:
            next_symptom_start_tau = min([symp_tau-tau if (tau<symp_tau) else np.inf for tau in active_tau])
            next_symptom_start_i = np.where(active_tau == symp_tau -next_symptom_start_tau)[0][0]
        else:
            next_symptom_start_tau = np.inf
            next_symptom_start_i = None
        if symptoms_indices != []:
            next_symptom_end_tau = min([symp_tau+7-tau if (tau<symp_tau+7) else np.inf for tau in active_tau])
            next_symptom_end_i = np.where(active_tau == symp_tau+7 -next_symptom_end_tau)[0][0]
        else:
            next_symptom_end_tau = np.inf
            next_symptom_end_i = None
            
        
        next_event_tau["start symptoms"] = next_symptom_start_tau
        next_event_tau["end symptoms"] = next_symptom_end_tau
        next_event_i["start symptoms"] = next_symptom_start_i
        next_event_i["end symptoms"] = next_symptom_end_i
        
        next_event_tau["pulse"] += -dt
        ################################################################################################################################################
        #Check if symptoms threshold is met
        if len(symptoms_indices)>=symp_thresh and not pulsed:
            next_event_tau["pulse"] = 1
            pulsed = True
        ################################################################################################################################################        
        #Record data
        t_v.append(t)
        A_v.append(A)
        I_v.append(len(infected_indices))
        
        #print(next_event_tau)
        #print(next_event_i)
        #print("Indices with symptoms:",symptoms_indices)
        #print("Indices with infection:",infected_indices)
        #print(active_tau)
        #print("t="+str(t)+"\n")
    t_v.append(t)
    A_v.append(A)
    I_v.append(len(infected_indices))
    return(t_v,A_v,I_v,t_inf,len(infected_indices)+len(recovered_indices))

In [133]:
R0 = 1.5
n =50
m = 1
Q_unsorted = np.random.exponential(1,n)
Q = np.sort(Q_unsorted)
[Q[k]<= (R0/n)*(k+m) for k in range(len(Q))]
P = np.random.uniform(0,1,n+m)
#Distribution parameters taken from ashcroft paper

weib_params_inf = [2.5573538434277925, 2.9506811454264685, -10.822291419176842, 9.946517003145942]
weib_params_test = [2.5573538434277925/2, 2.9506811454264685, -10.822291419176842, 9.946517003145942*1.15]

inf_cdf = lambda T1,T2: (1-np.exp(-((T2)/weib_params_inf[3])**weib_params_inf[1]))**weib_params_inf[0] - (1-np.exp(-((T1)/weib_params_inf[3])**weib_params_inf[1]))**weib_params_inf[0]
test_pdf = lambda T: weib_params_test[1]*((1-np.exp(-(T/weib_params_test[3])**weib_params_test[1]))**(weib_params_test[0]-1))*np.exp(-(T/weib_params_test[3])**weib_params_test[1])*(T/weib_params_test[3])**(weib_params_test[1]-1) if T>0 else 0
time_symptoms = -weib_params_inf[2]
symp_thresh = 3